The Heston model is a mathematical model used in quantitative finance to price options. It is named after its creator, Steven Heston. The Heston model is a stochastic volatility model that assumes that the volatility of the underlying asset follows a stochastic process, as opposed to the Black-Scholes model, which assumes that volatility is constant.

The Heston model consists of two stochastic differential equations (SDEs) that describe the evolution of the stock price and the volatility. The first SDE describes the evolution of the stock price:

$dS_t = \mu S_t dt + \sqrt{V_t} S_t dW_{1,t}$

where $S_t$ is the stock price at time $t$, $\mu$ is the expected return of the stock, $V_t$ is the volatility of the stock, and $W_{1,t}$ is a Wiener process.

The second SDE describes the evolution of the volatility:

$dV_t = \kappa(\theta - V_t) dt + \xi \sqrt{V_t} dW_{2,t}$

where $V_t$ is the volatility at time $t$, $\kappa$ is the rate at which the volatility reverts to its long-term mean $\theta$, $\theta$ is the long-term mean of the volatility, $\xi$ is the volatility of the volatility (also known as the "vol of vol"), and $W_{2,t}$ is another Wiener process.

The Heston model is used to price European call and put options. The price of an option under the Heston model is calculated using a Fourier transform of the characteristic function of the joint distribution of the stock price and the volatility. This makes the Heston model computationally more expensive than the Black-Scholes model, but it is more accurate in situations where the volatility of the underlying asset is not constant.

In [7]:
import numpy as np
from scipy.stats import norm

def heston_option_price(S, K, T, r, q, v0, kappa, theta, sigma, rho, n_sim, n_steps):
    # Define parameters
    S = 100
    K = 110
    T = 1
    r = 0.05
    q = 0
    v0 = 0.02
    kappa = 2
    theta = 0.02
    sigma = 0.3
    rho = -0.5
    n_sim = 100000
    n_steps = 100
    
    # Define constants
    dt = T / n_steps
    sqrtdt = np.sqrt(dt)
    rho1 = np.sqrt(1 - rho**2)
    # Generate random numbers
    Z1 = np.random.normal(size=(n_sim, n_steps))
    Z2 = rho * Z1 + rho1 * np.random.normal(size=(n_sim, n_steps))
    # Initialize arrays
    S_path = np.zeros((n_sim, n_steps + 1))
    v_path = np.zeros((n_sim, n_steps + 1))
    # Set initial values
    S_path[:, 0] = S
    v_path[:, 0] = v0
    # Generate paths
    for i in range(1, n_steps + 1):
        # Compute variance at current time step
        v_path[:, i] = np.maximum(0, v_path[:, i-1] + kappa * (theta - v_path[:, i-1]) * dt +
                                   sigma * np.sqrt(v_path[:, i-1]) * sqrtdt * Z1[:, i-1])
        # Compute stock price at current time step
        S_path[:, i] = S_path[:, i-1] * np.exp((r - q - 0.5 * v_path[:, i-1]) * dt +
                                               np.sqrt(v_path[:, i-1]) * sqrtdt * Z2[:, i-1])
    # Compute option prices
    payoff = np.maximum(0, S_path[:, -1] - K)
    option_price = np.exp(-r * T) * np.mean(payoff)
    return option_price



In [8]:



# Call function and assign output to variable
option_price = heston_option_price(S, K, T, r, q, v0, kappa, theta, sigma, rho, n_sim, n_steps)

# Print the result
print("The option price is:", option_price)


The option price is: 3.2164679068803843


This function takes the following inputs:

* S: current stock price
* K: option strike price
* T: option time to maturity (in years)
* r: risk-free interest rate
* q: dividend yield
* v0: initial variance of the Heston model
* kappa: mean reversion speed of the variance process
* theta: long-run variance level of the variance process
* sigma: volatility of the variance process
* rho: correlation coefficient between the stock price and the variance process
* n_sim: number of Monte Carlo simulations
* n_steps: number of time steps in the simulation

And it returns the option price estimated using Monte Carlo simulation